In [2]:
pip install pygame

Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np
import pygame
from time import time,sleep
from random import randint as r
import random
import pickle

pygame 2.1.2 (SDL 2.0.16, Python 3.8.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
 ### Description
# There are three designated locations in the grid world indicated by R(ed),
# G(reen), and B(lue). When the episode starts, the START is 
# at a random square and the EXIT is at a random square. The START path
# must pass by the TREASURE, from the TREASURE, the path leads to the EXIT. at the EXIT the episode ends. There are 4 obstacles called WALL.
# Map:
# S(tart), P(ath), T(reasure), W(all), E(xit)
# "4x4":["SPPW",
#        "PWPP",
#        "PTPW",
#        "PWPE"]

In [130]:
n = 4
scrx = n*100
scry = n*100
background = (51,51,51)
screen = pygame.display.set_mode((scrx,scry))
colors = [(51,51,51) for i in range(n**2)]
reward = np.zeros((n,n))
terminals = []
penalities = 4

# learning rate
alpha = 0.01

# weight measure of how important do we find future actions vs current action or future reward over current reward. values between 0 and 1
gamma = 0.9

current_pos = [0,0]

# values between 0 and 1. The higher the epsilon, the more likely we are to perform a random action
# overtime, we want to stop our model to stop exploring
epsilon = 0.25

# exit, I put the reward for the exit to be the highest one.
reward[n-1,n-1] = 2

#treasure, I put -1 for the treasure for the ball to pass by the value, but not stay there.
reward[2,1] = -1

# start
colors[0] = (0,255,0)

# exit
colors[n**2 - 1] = (0,255,0)

#treasure
colors[n**2 - 7] = (253,255,71)

terminals.append(n**2 - 1)



actions = {"up": 0,"down" : 1,"left" : 2,"right" : 3}



# list for accuracies
episode_rewards= []



while penalities != 0:
    i = r(0,n-1)
    j = r(0,n-1)
    # if the case in the reward array are equal to 0, we replace it by -5 and put penalties
    if reward[i,j] == 0 and [i,j] != [0,0] and [i,j] != [n-1,n-1] and [i,j]!=[1,2]:
        reward[i,j] = -5
        penalities -= 1
        colors[n*i+j] = (255,0,0)  #red
        terminals.append(n*i+j)

# initial q_table
Q = np.zeros((n**2,4))
states = {}
k = 0
for i in range(n):
    for j in range(n):
        states[(i,j)] = k
        k+=1


def layout():
    c = 0
    for i in range(0,scrx,100):
        for j in range(0,scry,100):
            pygame.draw.rect(screen,(255,255,255),(j,i,j+100,i+100),0)
            pygame.draw.rect(screen,colors[c],(j+3,i+3,j+95,i+95),0)
            c+=1


def select_action(current_state):
    global current_pos,epsilon
    possible_actions = []
    if np.random.uniform() <= epsilon:
        if current_pos[1] != 0:
            possible_actions.append("left")
        if current_pos[1] != n-1:
            possible_actions.append("right")
        if current_pos[0] != 0:
            possible_actions.append("up")
        if current_pos[0] != n-1:
            possible_actions.append("down")
        action = actions[possible_actions[r(0,len(possible_actions) - 1)]]
    else:
        m = np.min(Q[current_state])
        if current_pos[0] != 0:
            possible_actions.append(Q[current_state,0])
        else:
            possible_actions.append(m - 100)
        if current_pos[0] != n-1:
            possible_actions.append(Q[current_state,1])
        else:
            possible_actions.append(m - 100)
        if current_pos[1] != 0:
            possible_actions.append(Q[current_state,2])
        else:
            possible_actions.append(m - 100)
        if current_pos[1] != n-1:
            possible_actions.append(Q[current_state,3])
        else:
            possible_actions.append(m - 100)
        # action = np.argmax(possible_actions)
        action = random.choice([i for i,a in enumerate(possible_actions) if a == max(possible_actions)])
        return action
def episode():
    global current_pos,epsilon
    current_state = states[(current_pos[0],current_pos[1])]
    action = select_action(current_state)
    if action == 0:
        current_pos[0] -= 1
    elif action == 1:
        current_pos[0] += 1
    elif action == 2:
        current_pos[1] -= 1
    elif action == 3:
        current_pos[1] += 1
    new_state = states[(current_pos[0],current_pos[1])]
    if new_state not in terminals:
        # The core of the algorithm is a Bellman equation 
        # as a simple value iteration update, using the weighted average of the current value and the new information
        Q[current_state,action] += alpha*(reward[current_pos[0],current_pos[1]] + gamma*(np.max(Q[new_state])) - Q[current_state,action])
    else:
        Q[current_state,action] += alpha*(reward[current_pos[0],current_pos[1]] - Q[current_state,action])
        current_pos = [0,0]
        epsilon -= 1e-3


run = True
for i in range(000):
    episode()
while run:
    sleep(0.1)
    screen.fill(background)
    layout()
    pygame.draw.circle(screen,(230,25,129),(current_pos[1]*100 + 50,current_pos[0]*100 + 50),30,0)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            run = False
    pygame.display.flip()
    episode()

pygame.quit()
print(epsilon)
# moving_avg = np.convolve(episode_rewards, np.ones((SHOW_EVERY,))/SHOW_EVERY, mode="valid")
# plt.plot([i for i in range(len(moving_avg))], moving_avg)
# plt.ylabel(f"reward {SHOW_EVERY} ma")
# plt.xlabel("episode #")
# plt.show()

# f = open("Q.txt","w")
# f.write(pickle.dumps(Q))
# f.close()

0.243


In [3]:
states = {}
k = 0
for i in range(4):
    for j in range(4):
        states[(i,j)] = k
        k+=1

In [4]:
states

{(0, 0): 0,
 (0, 1): 1,
 (0, 2): 2,
 (0, 3): 3,
 (1, 0): 4,
 (1, 1): 5,
 (1, 2): 6,
 (1, 3): 7,
 (2, 0): 8,
 (2, 1): 9,
 (2, 2): 10,
 (2, 3): 11,
 (3, 0): 12,
 (3, 1): 13,
 (3, 2): 14,
 (3, 3): 15}

In [9]:
Q = np.zeros((4**2,4))
Q

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [11]:
terminals = []
terminals.append(4**2 - 1)
terminals

[15]

In [49]:
penalities = 4

reward = -np.ones((4,4))
# exit, I put the reward for the exit to be the highest one.
reward[4-1,4-1] = 2
#treasure, I put -1 for the treasure for the ball to pass by the value, but not stay there.
reward[2,1] = 1

colors = [(51,51,51) for i in range(4**2)]

while penalities != 0:
    i = r(0,3)
    j = r(0,3)
    # if the case in the reward array are equal to 0, we replace it by -5 and put penalties
    if reward[i,j] == -1 and [i,j] != [0,0] and [i,j] != [4-1,4-1] and [i,j]!=[1,2]:
        reward[i,j] = -5
        penalities -= 1
        colors[3*i+j] = (255,0,0)  #red
        terminals.append(3*i+j)

In [50]:
reward

array([[-1., -5., -5., -1.],
       [-1., -1., -1., -1.],
       [-1.,  1., -1., -1.],
       [-1., -5., -5.,  2.]])

In [51]:
terminals

[15,
 10,
 9,
 11,
 9,
 1,
 6,
 6,
 9,
 1,
 4,
 0,
 3,
 0,
 1,
 9,
 6,
 3,
 4,
 0,
 2,
 2,
 3,
 4,
 1,
 1,
 9,
 9,
 2,
 9,
 9,
 4,
 3,
 10,
 9,
 2,
 3,
 11,
 9,
 6,
 9,
 3,
 6,
 9,
 3,
 1,
 9,
 8,
 6,
 10,
 1,
 2,
 11]